### **ICESat-2数据处理**

In [ ]:
import h5py
import numpy as np
# import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from pyrsimg import readTiff, imgShow, second_to_dyr


1.6800000000000637

In [9]:
path_atl06 = 'data/laser/processed_ATL06_20200806013350_06370802_006_01.h5'
path_rsimg = 'data/rsimg/s2_gla_region_20220707.tif'


In [17]:
# !h5ls $path_atl06/gt1l
!h5ls $path_atl06/gt1l/land_ice_segments
# !h5ls $path_atl06/orbit_info


atl06_quality_summary    Dataset {1890/Inf}
bias_correction          Group
delta_time               Dataset {1890/Inf}
dem                      Group
fit_statistics           Group
geophysical              Group
ground_track             Group
h_li                     Dataset {1890/Inf}
h_li_sigma               Dataset {1890/Inf}
latitude                 Dataset {1890/Inf}
longitude                Dataset {1890/Inf}
segment_id               Dataset {1890/Inf}
sigma_geo_h              Dataset {1890/Inf}


### Visualization

#### 读取并合并多波束数据

In [18]:
vars_atl06 = {'lat':[], 'lon':[], 'time_delta':[], 'elev':[], 'qual':[]}
beams = ['gt1l','gt1r','gt2l','gt2r','gt3l','gt3r']
with h5py.File(path_atl06,'r') as atl06:
    for beam in beams:
        vars_atl06['lat'] += list(atl06[beam+'/land_ice_segments/latitude'][:])
        vars_atl06['lon'] += list(atl06[beam+'/land_ice_segments/longitude'][:])
        vars_atl06['time_delta'] += list(atl06[beam+'/land_ice_segments/delta_time'][:]) ## Seconds start with the reference sdp time(2018-01-01).
        vars_atl06['elev'] += list(atl06[beam+'/land_ice_segments/h_li'][:])
        ### quality
        vars_atl06['qual'] += list(atl06[beam+'/land_ice_segments/atl06_quality_summary'][:])
    vars_atl06['lat'] = np.array(vars_atl06['lat'])
    vars_atl06['lon'] = np.array(vars_atl06['lon'])
    vars_atl06['time_delta'] = np.array(vars_atl06['time_delta'])
    vars_atl06['elev'] = np.array(vars_atl06['elev'])
    vars_atl06['qual'] = np.array(vars_atl06['qual'])
    print(vars_atl06['lat'].shape)
    ## orbit information.
    vars_atl06['cycle'] = atl06['orbit_info/cycle_number'][0]
    vars_atl06['track'] = atl06['orbit_info/rgt'][0]
    vars_atl06['orient'] = atl06['orbit_info/sc_orient'][0]

## 2. remote sensing image
rsimg = readTiff(path_rsimg)


(10885,)


In [21]:
region = [94.56, 30.38, 95.04, 30.77]
processed_vars_atl06 = {}

ids = np.where((vars_atl06['lat']>region[1]) \
                    & (vars_atl06['lat']<region[3]) \
                    & (vars_atl06['lon']>region[0]) \
                    & (vars_atl06['lon']<region[2]) \
                    & (vars_atl06['elev']<9000) \
                    & (vars_atl06['qual']==0))
processed_vars_atl06['lat'] = vars_atl06['lat'][ids]
processed_vars_atl06['lon'] = vars_atl06['lon'][ids]
processed_vars_atl06['elev'] = vars_atl06['elev'][ids]
processed_vars_atl06['time_delta'] = vars_atl06['time_delta'][ids]

## orbit information
processed_vars_atl06['cycle'] = vars_atl06['cycle']
processed_vars_atl06['track'] = vars_atl06['track']
processed_vars_atl06['orient'] = vars_atl06['orient']


In [22]:
processed_vars_atl06


{'lat': array([30.38889779, 30.38907787, 30.38925794, ..., 30.76947773,
        30.76965776, 30.76983778], shape=(4269,)),
 'lon': array([94.83061495, 94.83059528, 94.83057554, ..., 94.8553529 ,
        94.85533286, 94.85531282], shape=(4269,)),
 'elev': array([2948.6948, 2950.6106, 2955.1477, ..., 3991.605 , 3991.4717,
        3991.2407], shape=(4269,), dtype=float32),
 'time_delta': array([81912883.01548333, 81912883.01832423, 81912883.02116695, ...,
        81912889.19227107, 81912889.1950888 , 81912889.19790927],
       shape=(4269,)),
 'cycle': np.int8(8),
 'track': np.int16(637),
 'orient': np.int8(0)}

In [24]:
### Apply time conversion (seconds to decimal year)
### Seconds (from 2018-01-01 00:00:00) to decimal year
print('转换前：',processed_vars_atl06['time_delta'][0:10])
# for beam in beams:
processed_vars_atl06['time_dyr'] = second_to_dyr(processed_vars_atl06['time_delta'], time_start = '2018-01-01 00:00:00')
print('转换后：',processed_vars_atl06['time_dyr'][0:10])


转换前： [81912883.01548333 81912883.01832423 81912883.02116695 81912883.02400978
 81912883.0268495  81912883.02968517 81912883.03251266 81912883.0353311
 81912883.0381452  81912883.04095736]
转换后： [2020.5958093 2020.5958093 2020.5958093 2020.5958093 2020.5958093
 2020.5958093 2020.5958093 2020.5958093 2020.5958093 2020.5958093]


## write out to .h5 file.

In [ ]:
# file_out = 'data/laser/isat2_gla_region.h5'
# with h5py.File(file_out, "w") as f_out:
#     [f_out.create_dataset(key, data=processed_vars_atl06[key]) for key in processed_vars_atl06.keys()]
#     print('written file:', (file_out))



written file: data/laser/isat2_gla_region.h5
